### Imports

In [16]:
import os
from getpass import getpass
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage

In [3]:
keys = ["TAVILY_API_KEY"]

for key in keys:
    if key not in os.environ:
        os.environ[key] = getpass(f"Enter your {key}: ")

In [12]:
#initialize model
llm = ChatOllama(model="gpt-oss:20b",)

In [13]:
@tool
def search_web(query: str) -> str:
    """Search the web for information"""
    if "TAVILY_API_KEY" not in os.environ:
        return "Web search unavailable (no Tavily API key)"

    try:
        search = TavilySearchResults(max_results=3)
        results = search.invoke(query)
        if results:
            return f"Search results for '{query}': {results[:200]}..."
        return "No results found"
    except Exception as e:
        return f"Search error: {str(e)}"

In [14]:
#create ReAct agent with tools
tools = [search_web]

In [15]:
react_agent = create_react_agent(model=llm, tools=tools)

In [17]:
result = react_agent.invoke({
    "messages": [HumanMessage(content="What is 25 * 16, and can you find information about LangGraph?")]
})

/var/folders/39/t3m199396g56pr72q2ngcmqc0000gn/T/ipykernel_14802/3885224549.py:8: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(max_results=3)


In [18]:
print("Agent conversation:")
for message in result["messages"]:
    if hasattr(message, 'content') and message.content:
        print(f"{type(message).__name__}: {message.content[:100]}...")
    elif hasattr(message, 'name'):
        print(f"Tool Result ({message.name}): {message.content[:50]}...")

Agent conversation:
HumanMessage: What is 25 * 16, and can you find information about LangGraph?...
Tool Result (None): ...
ToolMessage: Search results for 'LangGraph': [{'title': 'What is LangGraph?', 'url': 'https://www.ibm.com/think/t...
AIMessage: **25 × 16 = 400**

---

## What is **LangGraph**?

| Feature | Description |
|---------|------------...
